# Notebook Description: Fine-tuning Llama-3.2-1B with Unsloth for Math Problem Classification

This notebook demonstrates how to fine-tune a relatively small Large Language Model (unsloth/Llama-3.2-1B) for the math problem classification task using the Unsloth library for efficient training. The task is framed as instruction-following, where the model generates the textual category name based on the problem description. Data is sourced from [this](https://www.kaggle.com/competitions/classification-of-math-problems-by-kasut-academy/overview) Kaggle competition with some additional augmentation.

### What this Notebook has:

- Framework & Model: Utilizes the unsloth library to efficiently fine-tune the unsloth/Llama-3.2-1B model, leveraging techniques like 4-bit quantization and LoRA (Low-Rank Adaptation).
- Task Formulation: Treats classification as an instruction-following, text generation task. A specific prompt template is used, asking the model to classify the input math problem and generate the corresponding category name (e.g., "Algebra", "Number Theory") in the response section.
- Loads the original competition training data (train.csv) and an augmented dataset (augmented_train_cleaned.csv).
- Combines a subset of the original data (first 9189 samples) with the augmented data for the training process. The remaining 1000 original samples are used later for internal testing.
- Formats the combined dataset using the defined instruction prompt template.
- Applies LoRA adapters to the base model.
- Uses the SFTTrainer (Supervised Fine-tuning Trainer) from the trl library for training.
- Trains the model for a fixed number of steps (640) using specific hyperparameters like learning rate, batch size, gradient accumulation, and an 8-bit AdamW optimizer.
- Performs inference on the hold-out set (last 1000 samples of the original training data) using the fine-tuned model.
- Generates predictions (category names) for the official competition test set (test.csv) using the fine-tuned model and the instruction prompt format.
- Parses the generated text and maps the predicted category names back to numeric IDs required for submission, handling potential parsing failures by assigning a default label (0).
- Creates and saves the submission.csv file.
Model Saving: Saves the trained LoRA adapter weights and the tokenizer.

### Next steps for this exploration:

- Experiment with different base models available through Unsloth (e.g., larger Llama variants, Mistral models).
- Tune the LoRA configuration (rank r, lora_alpha, target_modules).
- Optimize the SFTTrainer hyperparameters (learning rate, training steps/epochs, batching strategy, scheduler).
- Refine the instruction prompt to potentially improve clarity and model performance.
- Enhance the response parsing logic to be more robust against variations in the model's output format.
- Analyze the impact of the augmented data compared to using only the original dataset.
- Compare the performance, training time, and resource requirements of this approach with the T5 and sequence classification (DistilBERT, DeBERTa) methods.

**Public Test Score:** 0.8346

# Install Packages

In [2]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

# Import Libraries

In [ ]:
import re
import torch
import pandas as pd
from datasets import load_dataset
from trl import SFTTrainer
from tqdm.auto import tqdm
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Load LLAMA 3.2 1B Model and Tokenizer

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # 4bit quantization to reduce memory usage. 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    #token = "" HF_Token for gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/scratch/huanggrp/prudhvi/nlp_project/unsloth-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla V100-SXM2-16GB. Num GPUs = 4. Max memory: 15.766 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.4.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


# Load Original Training Data and Merge with New Data

In [ ]:
label_map = {0: "Algebra",
            1: "Geometry and Trigonometry",
            2: "Calculus and Analysis",
            3: "Probability and Statistics",
            4: "Number Theory",
            5: "Combinatorics and Discrete Math",
            6: "Linear Algebra",
            7: "Abstract Algebra and Topology"}


train_actual = pd.read_csv("train.csv")
aug_train = pd.read_csv("augmented_train_cleaned.csv")
train = pd.concat([train_actual[:9189], aug_train]).reset_index(drop=True)
train["instruction"] = "Classify this math problem into one of these eight topics: Algebra, Geometry and Trigonometry, Calculus and Analysis, Probability and Statistics, Number Theory, Combinatorics and Discrete Math, Linear Algebra, Abstract Algebra and Topology."
train["label"] = train["label"].map(label_map)
train = train.rename(columns={"label": "output", "Question": "input"})
train.to_csv("train_updated.csv", index=False)


dataset = load_dataset("csv", data_files="train_updated.csv", split="train")

Generating train split: 20598 examples [00:00, 23439.12 examples/s]


Fine-tune on the first 9189 samples of the original training set and the augmented training set. And use the remaining 1000 samples for testing the model.

# Prepare Data

In [4]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 20598/20598 [00:01<00:00, 10426.92 examples/s]


In [5]:
dataset, dataset[0]

(Dataset({
     features: ['input', 'output', 'instruction', 'text'],
     num_rows: 20598
 }),
 {'input': "A solitaire game is played as follows.  Six distinct pairs of matched tiles are placed in a bag.  The player randomly draws tiles one at a time from the bag and retains them, except that matching tiles are put aside as soon as they appear in the player's hand.  The game ends if the player ever holds three tiles, no two of which match; otherwise the drawing continues until the bag is empty.  The probability that the bag will be emptied is $p/q,\\,$ where $p\\,$ and $q\\,$ are relatively prime positive integers.  Find $p+q.\\,$ ",
  'output': 'Probability and Statistics',
  'instruction': 'Classify this math problem into one of these eight topics: Algebra, Geometry and Trigonometry, Calculus and Analysis, Probability and Statistics, Number Theory, Combinatorics and Discrete Math, Linear Algebra, Abstract Algebra and Topology.',
  'text': "Below is an instruction that describes a ta

In [6]:
print(dataset[0]["text"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify this math problem into one of these eight topics: Algebra, Geometry and Trigonometry, Calculus and Analysis, Probability and Statistics, Number Theory, Combinatorics and Discrete Math, Linear Algebra, Abstract Algebra and Topology.

### Input:
A solitaire game is played as follows.  Six distinct pairs of matched tiles are placed in a bag.  The player randomly draws tiles one at a time from the bag and retains them, except that matching tiles are put aside as soon as they appear in the player's hand.  The game ends if the player ever holds three tiles, no two of which match; otherwise the drawing continues until the bag is empty.  The probability that the bag will be emptied is $p/q,\,$ where $p\,$ and $q\,$ are relatively prime positive integers.  Find $p+q.\,$ 

### Response:
Probability and Statistics<

# Setup Trainer

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 640,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 20598/20598 [00:06<00:00, 3085.91 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


# Show current memory stats

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla V100-SXM2-16GB. Max memory = 15.766 GB.
1.457 GB of memory reserved.


# Start Training

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,598 | Num Epochs = 4 | Total steps = 640
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 8 x 1) = 128
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.180300
2,2.206900
3,2.201500
4,2.052200
5,1.921500
6,1.725000
7,1.569800
8,1.450400
9,1.232700
10,1.096500


# Show final memory and time stats

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2863.9588 seconds used for training.
47.73 minutes used for training.
Peak reserved memory = 15.01 GB.
Peak reserved memory for training = 13.553 GB.
Peak reserved memory % of max memory = 95.205 %.
Peak reserved memory for training % of max memory = 85.963 %.


In [39]:
train_actual = pd.read_csv("train.csv")

# Inference on Remaining 1000 samples

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

train_actual["instruction"] = "Classify this math problem into one of these eight topics: Algebra, Geometry and Trigonometry, Calculus and Analysis, Probability and Statistics, Number Theory, Combinatorics and Discrete Math, Linear Algebra, Abstract Algebra and Topology."
train_actual.rename(columns = {"Question": "input", "label": "output"}, inplace=True)


raw_outputs = []
for i in tqdm(range(9189,len(train_actual))):
  inputs = tokenizer(
  [
      prompt.format(
          train_actual.iloc[0]["instruction"], # instruction
          train_actual.iloc[i]["input"], # input
          "", # output - blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  raw_outputs.append(tokenizer.batch_decode(outputs))

100%|██████████| 1000/1000 [02:18<00:00,  7.22it/s]


In [41]:
print(raw_outputs[0][0], train_actual.iloc[9189]["output"])

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify this math problem into one of these eight topics: Algebra, Geometry and Trigonometry, Calculus and Analysis, Probability and Statistics, Number Theory, Combinatorics and Discrete Math, Linear Algebra, Abstract Algebra and Topology.

### Input:
9. (18 points) A square piece of paper, if folded 2 times, folded into a small square, and a notch is cut on each side of this small square, when the paper is unfolded, there are 4 small holes in the middle of the paper (notches on the edges do not count). A square piece of paper, if folded 6 times into a small square, and a notch is cut on each side of this small square, and then the paper is unfolded, there will be $\qquad$ small holes in the middle of the paper.

### Response:
Combinatorics and Discrete Math<|end_of_text|> 5


In [42]:
test_set = train_actual[9189:]
test_set["raw_outputs"] = [raw_output[0] for raw_output in raw_outputs]
test_set

/tmp/ipykernel_2289039/4127684203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set["raw_outputs"] = [raw_output[0] for raw_output in raw_outputs]


,input,output,instruction,raw_outputs
9189,"9. (18 points) A square piece of paper, if fol...",5,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
9190,"Find all triplets $ (x,y,z) $ of positive inte...",4,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
9191,"Let $A$ be the set $\{k^{19}-k: 1<k<20, k\in N...",4,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
9192,8. We consider all possible sets consisting of...,5,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
9193,For what value of $x$ will $\frac{2x-1}{2x+2}$...,0,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
...,...,...,...,...
10184,Is there a first- or second-degree polynomial ...,0,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
10185,4. Let the side lengths of $\triangle A B C$ b...,1,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
10186,A triangular array of squares has one square i...,5,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
10187,Suppose we are given seven points that are equ...,1,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...


# Parse Model Responses

In [ ]:
def parse_output(output):
    re_match = re.search(r'### Response:\n(.*?)<\|end_of_text\|>', output, re.DOTALL)
    if re_match:
        response = re_match.group(1).strip()
        return response
    else:
        return ''

test_set["parsed_outputs"] = test_set["raw_outputs"].apply(parse_output)
test_set

/tmp/ipykernel_2289039/1755018042.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set["parsed_outputs"] = test_set["raw_outputs"].apply(parse_output)


,input,output,instruction,raw_outputs,parsed_outputs
9189,"9. (18 points) A square piece of paper, if fol...",5,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Combinatorics and Discrete Math
9190,"Find all triplets $ (x,y,z) $ of positive inte...",4,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory
9191,"Let $A$ be the set $\{k^{19}-k: 1<k<20, k\in N...",4,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory
9192,8. We consider all possible sets consisting of...,5,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Combinatorics and Discrete Math
9193,For what value of $x$ will $\frac{2x-1}{2x+2}$...,0,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra
...,...,...,...,...,...
10184,Is there a first- or second-degree polynomial ...,0,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra
10185,4. Let the side lengths of $\triangle A B C$ b...,1,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry
10186,A triangular array of squares has one square i...,5,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Combinatorics and Discrete Math
10187,Suppose we are given seven points that are equ...,1,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry


In [44]:
test_set["output"] = test_set["output"].map(label_map)
test_set

/tmp/ipykernel_2289039/1651769868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set["output"] = test_set["output"].map(label_map)


,input,output,instruction,raw_outputs,parsed_outputs
9189,"9. (18 points) A square piece of paper, if fol...",Combinatorics and Discrete Math,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Combinatorics and Discrete Math
9190,"Find all triplets $ (x,y,z) $ of positive inte...",Number Theory,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory
9191,"Let $A$ be the set $\{k^{19}-k: 1<k<20, k\in N...",Number Theory,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory
9192,8. We consider all possible sets consisting of...,Combinatorics and Discrete Math,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Combinatorics and Discrete Math
9193,For what value of $x$ will $\frac{2x-1}{2x+2}$...,Algebra,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra
...,...,...,...,...,...
10184,Is there a first- or second-degree polynomial ...,Algebra,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra
10185,4. Let the side lengths of $\triangle A B C$ b...,Geometry and Trigonometry,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry
10186,A triangular array of squares has one square i...,Combinatorics and Discrete Math,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Combinatorics and Discrete Math
10187,Suppose we are given seven points that are equ...,Geometry and Trigonometry,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry


# Calculate Accuracy on Test Set

In [45]:
print("Accuracy: ", sum(test_set["output"] == test_set["parsed_outputs"]))

Accuracy:  851


# Save Model (Just LoRA Adapters) and Tokenzer

In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

# Make Predictions on Comptetion Test Set

In [47]:
public_set = pd.read_csv("test.csv")
public_set

,id,Question
0,0,b'Solve 0 = -i - 91*i - 1598*i - 64220 for i.\n'
1,1,Galperin G.A.\n\nA natural number $N$ is 999.....
2,2,Example 7 Calculate $\frac{1}{2 \sqrt{1}+\sqrt...
3,3,"If $A$, $B$, and $C$ represent three distinct ..."
4,4,2. Calculate $1+12+123+1234+12345+123456+12345...
...,...,...
3039,3039,"Find the greatest possible value of $pq + r$, ..."
3040,3040,"4. Given that $a, b, c$ are the lengths of the..."
3041,3041,"3.18. Find the eccentricity, coordinates of th..."
3042,3042,Find the least positive integer $k$ for which ...


In [ ]:
FastLanguageModel.for_inference(model)

public_set["instruction"] = "Classify this math problem into one of these eight topics: Algebra, Geometry and Trigonometry, Calculus and Analysis, Probability and Statistics, Number Theory, Combinatorics and Discrete Math, Linear Algebra, Abstract Algebra and Topology."
public_set.rename(columns = {"Question": "input"}, inplace=True)

raw_outputs = []
for i in tqdm(range(len(public_set))):
  inputs = tokenizer(
  [
      prompt.format(
          public_set.iloc[0]["instruction"], 
          public_set.iloc[i]["input"], 
          "",
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  raw_outputs.append(tokenizer.batch_decode(outputs))

 38%|███▊      | 1146/3044 [02:35<05:08,  6.14it/s]Unsloth: Input IDs of length 2135 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
100%|██████████| 3044/3044 [06:58<00:00,  7.27it/s]


In [49]:
public_set["raw_outputs"] = [raw_output[0] for raw_output in raw_outputs]
public_set

,id,input,instruction,raw_outputs
0,0,b'Solve 0 = -i - 91*i - 1598*i - 64220 for i.\n',Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
1,1,Galperin G.A.\n\nA natural number $N$ is 999.....,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
2,2,Example 7 Calculate $\frac{1}{2 \sqrt{1}+\sqrt...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
3,3,"If $A$, $B$, and $C$ represent three distinct ...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
4,4,2. Calculate $1+12+123+1234+12345+123456+12345...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
...,...,...,...,...
3039,3039,"Find the greatest possible value of $pq + r$, ...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
3040,3040,"4. Given that $a, b, c$ are the lengths of the...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
3041,3041,"3.18. Find the eccentricity, coordinates of th...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...
3042,3042,Find the least positive integer $k$ for which ...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...


In [50]:
public_set["parsed_outputs"] = public_set["raw_outputs"].apply(parse_output)
public_set

,id,input,instruction,raw_outputs,parsed_outputs
0,0,b'Solve 0 = -i - 91*i - 1598*i - 64220 for i.\n',Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra
1,1,Galperin G.A.\n\nA natural number $N$ is 999.....,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory
2,2,Example 7 Calculate $\frac{1}{2 \sqrt{1}+\sqrt...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Calculus and Analysis
3,3,"If $A$, $B$, and $C$ represent three distinct ...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra
4,4,2. Calculate $1+12+123+1234+12345+123456+12345...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Combinatorics and Discrete Math
...,...,...,...,...,...
3039,3039,"Find the greatest possible value of $pq + r$, ...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory
3040,3040,"4. Given that $a, b, c$ are the lengths of the...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry
3041,3041,"3.18. Find the eccentricity, coordinates of th...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry
3042,3042,Find the least positive integer $k$ for which ...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory


In [51]:
label2id = {v:k for k,v in label_map.items()}
label2id

{'Algebra': 0,
 'Geometry and Trigonometry': 1,
 'Calculus and Analysis': 2,
 'Probability and Statistics': 3,
 'Number Theory': 4,
 'Combinatorics and Discrete Math': 5,
 'Linear Algebra': 6,
 'Abstract Algebra and Topology': 7}

In [53]:
public_set["label"] = public_set["parsed_outputs"].map(label2id)
public_set

,id,input,instruction,raw_outputs,parsed_outputs,label
0,0,b'Solve 0 = -i - 91*i - 1598*i - 64220 for i.\n',Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra,0.0
1,1,Galperin G.A.\n\nA natural number $N$ is 999.....,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory,4.0
2,2,Example 7 Calculate $\frac{1}{2 \sqrt{1}+\sqrt...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Calculus and Analysis,2.0
3,3,"If $A$, $B$, and $C$ represent three distinct ...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra,0.0
4,4,2. Calculate $1+12+123+1234+12345+123456+12345...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Combinatorics and Discrete Math,5.0
...,...,...,...,...,...,...
3039,3039,"Find the greatest possible value of $pq + r$, ...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory,4.0
3040,3040,"4. Given that $a, b, c$ are the lengths of the...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry,1.0
3041,3041,"3.18. Find the eccentricity, coordinates of th...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry,1.0
3042,3042,Find the least positive integer $k$ for which ...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory,4.0


# Make Submission to the Competition

In [55]:
public_set["label"] = public_set["label"].fillna(0).astype(int)
public_set.rename(columns = {"input": "Question"})
public_set

,id,input,instruction,raw_outputs,parsed_outputs,label
0,0,b'Solve 0 = -i - 91*i - 1598*i - 64220 for i.\n',Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra,0
1,1,Galperin G.A.\n\nA natural number $N$ is 999.....,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory,4
2,2,Example 7 Calculate $\frac{1}{2 \sqrt{1}+\sqrt...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Calculus and Analysis,2
3,3,"If $A$, $B$, and $C$ represent three distinct ...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Algebra,0
4,4,2. Calculate $1+12+123+1234+12345+123456+12345...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Combinatorics and Discrete Math,5
...,...,...,...,...,...,...
3039,3039,"Find the greatest possible value of $pq + r$, ...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory,4
3040,3040,"4. Given that $a, b, c$ are the lengths of the...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry,1
3041,3041,"3.18. Find the eccentricity, coordinates of th...",Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Geometry and Trigonometry,1
3042,3042,Find the least positive integer $k$ for which ...,Classify this math problem into one of these e...,<|begin_of_text|>Below is an instruction that ...,Number Theory,4


In [58]:
public_set.rename(columns = {"input": "Question"}, inplace=True)

In [60]:
public_set[["id", "label"]].to_csv("submission.csv", index=False)
pd.read_csv("submission.csv")

,id,label
0,0,0
1,1,4
2,2,2
3,3,0
4,4,5
...,...,...
3039,3039,4
3040,3040,1
3041,3041,1
3042,3042,4
